# 🍛 Dapoer-AI: Agentic Chatbot untuk Rekomendasi Resep Masakan Indonesia

Notebook ini mendemonstrasikan implementasi Agentic Chatbot berbasis Langchain-style architecture dengan integrasi LLM (Gemini) dan dataset resep masakan Indonesia.

Tujuan utama chatbot ini adalah:
- Memberikan rekomendasi resep masakan berdasarkan **nama**, **bahan**, atau **metode memasak**
- Menyediakan opsi masakan yang **mudah** dibuat
- Menggunakan teknik **RAG-like** (Retrieval-Augmented Generation) agar respons tetap kontekstual meskipun input tidak eksplisit

Model yang digunakan: `Gemini 1.5 Flash`  
Dataset: [Indonesian_Food_Recipes.csv](https://raw.githubusercontent.com/audreeynr/dapoer-ai/refs/heads/main/data/Indonesian_Food_Recipes.csv)


## 1. Instalasi dan Import Library

In [1]:
!pip install -q google-generativeai

In [6]:
import pandas as pd
import re
import google.generativeai as genai
import google.generativeai as gemini
from google.colab import userdata

## 2. Konfigurasi API Gemini

API key diambil dari environment Google Colab secara aman melalui `userdata`, kemudian digunakan untuk menginisialisasi model Gemini.

In [7]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
if not GOOGLE_API_KEY:
    print("Error: GEMINI API key tidak ditemukan.")
    exit()

gemini.configure(api_key=GOOGLE_API_KEY)
model = gemini.GenerativeModel(model_name='gemini-1.5-flash-latest')

## 3. Load Dataset dan Pembersihan Data

Dataset `Indonesian_Food_Recipes.csv` diambil dari GitHub. Selanjutnya dilakukan:
- Menghapus baris kosong
- Menghilangkan duplikasi

In [10]:
CSV_FILE_PATH = 'https://raw.githubusercontent.com/audreeynr/dapoer-ai/refs/heads/main/data/Indonesian_Food_Recipes.csv'
df = pd.read_csv(CSV_FILE_PATH)
df_cleaned = df.dropna(subset=['Title', 'Ingredients', 'Steps']).drop_duplicates()

## 4. Normalisasi Teks

Agar pencarian menjadi fleksibel, seluruh teks diubah menjadi huruf kecil dan dihapus tanda baca/tambahan karakter menggunakan regex.

In [11]:
def normalize_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^a-z0-9\s]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return text

df_cleaned['Title_Normalized'] = df_cleaned['Title'].apply(normalize_text)
df_cleaned['Ingredients_Normalized'] = df_cleaned['Ingredients'].apply(normalize_text)
df_cleaned['Steps_Normalized'] = df_cleaned['Steps'].apply(normalize_text)


## 5. Format Output Resep

Fungsi `format_recipe()` menyusun hasil dalam bentuk yang lebih manusiawi dan mudah dibaca dengan markdown bullet dan struktur yang rapi.

In [12]:
def format_recipe(row):
    bahan_raw = re.split(r'\n|--|,', row['Ingredients'])
    bahan_list = [b.strip().capitalize() for b in bahan_raw if b.strip()]
    bahan_md = "\n".join([f"- {b}" for b in bahan_list])
    langkah_md = row['Steps'].strip()
    return f"""🍽 *{row['Title']}*

*Bahan-bahan:*
{bahan_md}

*Langkah Memasak:*
{langkah_md}"""

## 6. Fungsi Utama Agentic Chatbot

Fungsi `handle_user_query()` bertugas mendeteksi niat user dan menjalankan salah satu dari 5 tools berikut:

1. **Tool 1**: Pencarian berdasarkan nama resep
2. **Tool 2**: Pencarian berdasarkan bahan
3. **Tool 3**: Pencarian berdasarkan metode memasak
4. **Tool 4**: Filter resep mudah (steps < 300 karakter)
5. **Tool 5**: Retrieval-Augmented Generation (RAG-like) menggunakan LLM Gemini


In [16]:
def handle_user_query(prompt, model):
    prompt_lower = normalize_text(prompt)

    # Tool 1: Cari berdasarkan nama masakan
    match_title = df_cleaned[df_cleaned['Title_Normalized'].str.contains(prompt_lower)]
    if not match_title.empty:
        return format_recipe(match_title.iloc[0])

    # Tool 2: Cari berdasarkan bahan
    match_bahan = df_cleaned[df_cleaned['Ingredients_Normalized'].str.contains(prompt_lower)]
    if not match_bahan.empty:
        hasil = match_bahan.head(5)['Title'].tolist()
        return "Masakan yang menggunakan bahan tersebut:\n- " + "\n- ".join(hasil)

    # Tool 3: Cari berdasarkan metode masak
    for metode in ['goreng', 'panggang', 'rebus', 'kukus']:
        if metode in prompt_lower:
            cocok = df_cleaned[df_cleaned['Steps_Normalized'].str.contains(metode)]
            if not cocok.empty:
                hasil = cocok.head(5)['Title'].tolist()
                return f"Masakan yang dimasak dengan cara {metode}:\n- " + "\n- ".join(hasil)

    # Tool 4: Filter kesulitan
    if "mudah" in prompt_lower or "pemula" in prompt_lower:
        hasil = df_cleaned[df_cleaned['Steps'].str.len() < 300].head(5)['Title'].tolist()
        return "Rekomendasi masakan mudah:\n- " + "\n- ".join(hasil)

    # Tool 5: RAG-like (dengan prompt masukan)
    docs = "\n\n".join([
        f"{row['Title']}:\nBahan: {row['Ingredients']}\nLangkah: {row['Steps']}"
        for _, row in df_cleaned.sample(5, random_state=42).iterrows()
    ])
    full_prompt = f"""
Berikut beberapa resep masakan Indonesia:

{docs}

Gunakan referensi di atas untuk menjawab pertanyaan berikut:
{prompt}
"""
    response = model.generate_content(full_prompt)
    return response.text

## 7. Interaksi Langsung dengan Chatbot

Kode berikut akan membuka sesi interaktif di mana pengguna bisa bertanya tentang resep makanan, dan chatbot akan merespons secara dinamis menggunakan tool yang relevan.

In [22]:
while True:
    user_input = input("Tanyakan resep, bahan hingga ide masakan (atau ketik 'exit'): ")
    if user_input.lower() == 'exit':
        break
    print("\n" + handle_user_query(user_input, model) + "\n" + "="*60 + "\n")

Tanyakan resep, bahan hingga ide masakan (atau ketik 'exit'): aku ingin makanan bergizi, ada ide?

Berdasarkan resep-resep di atas,  untuk mendapatkan makanan bergizi, saya sarankan kombinasi beberapa resep atau memodifikasi resep yang ada:

**Ide 1 (Kombinasi):**

* **Nasi Biryani (modifikasi):**  Meskipun menggunakan daging kambing,  Anda bisa mengurangi jumlahnya dan menambahkan sayuran seperti wortel, buncis, atau kacang polong ke dalam nasi biryani untuk meningkatkan serat dan nutrisi.  Penggunaan beras basmati juga lebih baik daripada beras biasa.
* **Salmon Salad ala Pay:**  Salad ini sudah cukup bergizi karena mengandung protein dari salmon, sayuran hijau (bayam), dan beberapa sayuran lainnya.  Anda bisa menambahkan lebih banyak sayuran seperti selada, mentimun, atau paprika untuk menambah variasi nutrisi.

Kombinasi ini memberikan sumber protein (daging kambing/salmon), karbohidrat kompleks (nasi basmati), dan berbagai vitamin serta mineral dari sayuran.


**Ide 2 (Modifikasi 